Google API YT v3 documentation

https://developers.google.com/youtube/v3/docs/

Code Python code samples

https://github.com/youtube/api-samples

In [16]:
import json
import pprint
import time
from datetime import datetime

import requests
import urllib.request
from bs4 import BeautifulSoup

import utility as util

import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

google_secrets = "../.secrets/google.json"
with open(google_secrets) as f:
    GOOGLE_API_KEY = json.load(f)["key"]

def get_children_names(x):
    out = []
    for c in x.children:
        if not c.name:
            pass
        elif not c.prefix:
            out.append(c.name)
        else:
            out.append(f"{c.prefix}:{c.name}")
    return out

def query_to_str(query):
    o = []
    for k, v in query.items():
        if v is None:
            continue
        if isinstance(v, list):
            v = ",".join(v)
        else:
            v = str(v)
        k = str(k)
        o.append(k + "=" + v)
    return "&".join(o)

def QueryMaker(object):
    # TODO: is this useful?
    HOST = "www.googleapis.com"
    
    @staticmethod
    def dict_to_str(query):
        return "&".join(["=".join(x) for x in query.items()])
    
    def __init__(self, api_service_name, api_version):
        self.api_service_name = api_service_name
        self.api_version = api_version
    
    def get(self, resource, id, part=[]):
        # TODO: not done
        query = self.dict_to_str(query)
        query = f"https://{host}/{path}?{query}"
        try:
            res = requests.get(query)
            print("Status code:", res.status_code)
            return json.loads(res.content)
        except Exception as e:
            print("Exception:", e)

def YTQueryMaker(QueryMaker):
    def __init__(self):
        super().__init__("youtube", "v3")
    
pp = pprint.PrettyPrinter(indent=4)

In [28]:
# Get details about a video given video ID
# Based on:
# https://developers.google.com/youtube/v3/getting-started
# See:
# https://stackoverflow.com/questions/48857015/how-do-i-add-an-api-key-to-this-youtube-data-api-request
host = "www.googleapis.com"
path = "youtube/v3/videos"
query = {
    "key": GOOGLE_API_KEY,
    "id": "kO9HmhwGzXs",
    "part": "snippet,contentDetails,statistics,status",
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

print()
print("===Content===")
item = payload["items"][0]
videoId = item["id"]
snippet = item["snippet"]
title = snippet["title"]
description = snippet["description"]
channelTitle = snippet["channelTitle"]
tags = snippet["tags"]

print("video ID:", videoId)
print("title:", title)
print("description:")
print("[[[", description.replace("\n", " ")[:200], "]]]")
print("channel title:", channelTitle)
print("tags:", *tags)
print("statistics:")
for k, v in item["statistics"].items():
    print("    ", k, v)

Status code: 200

===Content===
video ID: kO9HmhwGzXs
title: An Incorrect Summary of Elden Ring | Part 1
description:
[[[ Part 1: The Moon and Stars ━━━━━━━━━━━━━━━━━━━━━━━━━━ Follow me on Twitter! https://twitter.com/realMax0r Patreon: https://www.patreon.com/max0r Join the Max0r Discord! https://discord.gg/max0r Twitch ]]]
channel title: Max0r
tags: maxor max0r elden ring maxor elden ring maxor ring maxor dark souls max0r dark souls elden ring sseth SsethTzeentach Uberdanger Martincitopants
statistics:
     viewCount 1881815
     likeCount 142163
     favoriteCount 0
     commentCount 11043


In [16]:
# Get infomation about channel given channel ID
# See:
# https://developers.google.com/youtube/v3/docs/channels
host = "www.googleapis.com"
path = "youtube/v3/channels"
query = {
    "key": GOOGLE_API_KEY,
    "id": "UCsXV4t9YfUeDuZ276hE20Jg",
    "part": [
#         "auditDetails", # 403 - The caller does not have permission
        "brandingSettings",
        "contentDetails",
        "contentOwnerDetails",
        "id",
        "localizations",
        "snippet",
        "statistics",
        "status",
        "topicDetails",
    ]
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

print()
print("===Content===")
item = payload["items"][0]
print("Statistics:")
for k, v in item["statistics"].items():
    print("    ", k, v)
relatedPlaylists = item["contentDetails"]["relatedPlaylists"]
print("relatedPlaylists:")
for k, v in relatedPlaylists.items():
    print("    ", k, v)
snippet = item["snippet"]
title = snippet["title"]
description = snippet["description"]
print("title:", title)
print("description:")
print(description.replace("\n", " ")[:200])
topicDetails = item["topicDetails"]
print("topicDetails:")
for k, v in topicDetails.items():
    print("    ", k, v)

Status code: 200

===Content===
Statistics:
     viewCount 42868806
     subscriberCount 372000
     hiddenSubscriberCount False
     videoCount 346
relatedPlaylists:
     likes 
     uploads UUsXV4t9YfUeDuZ276hE20Jg
title: Adiyogi
description:
Yogi, mystic and visionary, Sadhguru is a spiritual master with a difference. An arresting blend of profundity and pragmatism, his life and work serves as a reminder that yoga is a contemporary scienc
topicDetails:
     topicIds ['/m/098wr', '/m/019_rr', '/m/06bvp']
     topicCategories ['https://en.wikipedia.org/wiki/Society', 'https://en.wikipedia.org/wiki/Lifestyle_(sociology)', 'https://en.wikipedia.org/wiki/Religion']


In [7]:
# get information of multiple channels in one request
host = "www.googleapis.com"
path = "youtube/v3/channels"
query = {
    "key": GOOGLE_API_KEY,
    "id": [
        "UCRLQIvQcoF41xZdrtKMXSvg",
        "UCoydhtfFSk1fZXNRnkGnneQ",
        "UC1pdbN6j_txoNhXVWVtouuQ",
        "UCm22FAXZMw1BaWeFszZxUKw"
    ],
    "part": [
        "contentDetails",
        "contentOwnerDetails",
        "id",
        "snippet",
        "statistics",
        "status",
        "topicDetails",
    ]
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

should_show_details = True
print()
print("===Content===")
items = payload["items"]
for item in items:
    snippet = item["snippet"]
    title = snippet["title"]
    description = snippet["description"]
    description = description.replace("\n", " ")[:100]
    print("title:", title)
    print("description:", description)
    topicDetails = item["topicDetails"]
    if should_show_details:
        print("topicDetails:")
        for k, v in topicDetails.items():
            print("    ", k, v)
        print("Statistics:")
        for k, v in item["statistics"].items():
            print("    ", k, v)
        relatedPlaylists = item["contentDetails"]["relatedPlaylists"]
        print("relatedPlaylists:")
        for k, v in relatedPlaylists.items():
            print("    ", k, v)
    print()

Status code: 200

===Content===
title: Kitboga
description: Everyday there are scammers taking advantage of people. I call them to waste their time, walk people
topicDetails:
     topicIds ['/m/019_rr', '/m/07c1v', '/m/02jjt']
     topicCategories ['https://en.wikipedia.org/wiki/Lifestyle_(sociology)', 'https://en.wikipedia.org/wiki/Technology', 'https://en.wikipedia.org/wiki/Entertainment']
Statistics:
     viewCount 346501202
     subscriberCount 2350000
     hiddenSubscriberCount False
     videoCount 442
relatedPlaylists:
     likes 
     uploads UUm22FAXZMw1BaWeFszZxUKw

title: ESOTERICA
description: I'm Dr. Justin Sledge and welcome to Esoterica:  Where we explore the arcane in history, philosophy,
topicDetails:
     topicIds ['/m/06bvp', '/m/019_rr', '/m/098wr', '/m/01k8wb']
     topicCategories ['https://en.wikipedia.org/wiki/Religion', 'https://en.wikipedia.org/wiki/Lifestyle_(sociology)', 'https://en.wikipedia.org/wiki/Society', 'https://en.wikipedia.org/wiki/Knowledge']
Stat

In [149]:
# Get videos from the (upload) playlist
# We can't query using id (this is playlist item ID). Must use playlistId
# We got the uploads playlist ID by calling channels.list
# A playlist can be accessed from the YT website via
# https://www.youtube.com/playlist?list=UURLQIvQcoF41xZdrtKMXSvg
host = "www.googleapis.com"
path = "youtube/v3/playlistItems"
query = {
    "key": GOOGLE_API_KEY,
    "playlistId": "UURLQIvQcoF41xZdrtKMXSvg",
    "maxResults": 20,
    "part": [
        "id", "contentDetails", "snippet", "status"
    ]
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

print()
print("===Content===")
print("pageInfo:")
for k, v in payload["pageInfo"].items():
    print("    ", k, v)
print("prevPageToken:", payload.get("prevPageToken", None))
print("nextPageToken:", payload.get("nextPageToken", None))
    
# each playlist item result (in items) is one of
# https://developers.google.com/resources/api-libraries/documentation/youtube/v3/csharp/latest/classGoogle_1_1Apis_1_1YouTube_1_1v3_1_1Data_1_1PlaylistItem.html
print()
print("===Item 0===")
item = payload["items"][0]
playlistItemId = item["id"]
privacyStatus = item["status"]["privacyStatus"]
print("playlistItemId:", playlistItemId)
print("privacyStatus:", privacyStatus)
contentDetails = item["contentDetails"]
videoId = contentDetails["videoId"]
videoPublishedAt = datetime.strptime(contentDetails["videoPublishedAt"], "%Y-%m-%dT%H:%M:%SZ")
print("videoId:", videoId)
print("videoPublishedAt:", videoPublishedAt)
snippet = item["snippet"]
title = snippet["title"]
description = snippet["description"].replace("\n", " ")[:200]
print("title:", title)
print("description:", description)
# redundant information
videoId = snippet["resourceId"]["videoId"]
videoPublishedAt = datetime.strptime(snippet["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
channelTitle = snippet["videoOwnerChannelTitle"]
channelId = snippet["videoOwnerChannelId"]
print("From channel", channelTitle, ", channelId:", channelId)

Status code: 200

===Content===
pageInfo:
     totalResults 313
     resultsPerPage 20
prevPageToken: None
nextPageToken: EAAaBlBUOkNCUQ

===Item 0===
playlistItemId: VVVSTFFJdlFjb0Y0MXhaZHJ0S01YU3ZnLnlXcVg4d281YWxR
privacyStatus: public
videoId: yWqX8wo5alQ
videoPublishedAt: 2022-04-01 19:40:18
title: Woman Left Child, Husband, Family Because of U.G. She Says. Hear His Reply...
description: "I'm not responsible."  Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage to Stand Alone: Conversations with U.G. Krishnamurti - https://amzn.to/3enKIiU This Dog Barking
From channel Lectures Beyond Beyond , channelId: UCRLQIvQcoF41xZdrtKMXSvg


In [154]:
# Get all videos from the (upload) playlist
pageToken = None
count = 0
items = []
while True:
    host = "www.googleapis.com"
    path = "youtube/v3/playlistItems"
    query = {
        "key": GOOGLE_API_KEY,
        "playlistId": "UURLQIvQcoF41xZdrtKMXSvg",
        "maxResults": 50,
        "part": "snippet",
        "pageToken": pageToken
    }
    query = query_to_str(query)
    query = f"https://{host}/{path}?{query}"
    try:
        res = requests.get(query)
        payload = json.loads(res.content)
        items.extend(payload["items"])
        count += 1
        print("o ", end=" ")
        try:
            pageToken = payload["nextPageToken"]
        except KeyError:
            break    
    except Exception as e:
        print("Exception:", e)

print()
print(f"Made {count} requests")
print(f"Got {len(items)} videos")

o  o  o  o  o  o  o  
Made 7 requests
Got 313 videos


In [155]:
# List all the videos we go earlier, and print the links to the videos
should_print_link = True
for item in items:
    snippet = item["snippet"]
    videoId = snippet["resourceId"]["videoId"]
    title = snippet["title"]
    description = snippet["description"]
    description = description.replace("\n"," ")[:80]
    if should_print_link:
        print(f"https://youtu.be/{videoId}", end=" ")
        print(f", title: [ {title} ]")
    else:
        print(f"videoId: {videoId} , title: [ {title} ]")
    print("    ", description)
    print()

https://youtu.be/yWqX8wo5alQ , title: [ Woman Left Child, Husband, Family Because of U.G. She Says. Hear His Reply... ]
     "I'm not responsible."  Books on U.G. Krishnamurti: Mind is a Myth - https://amz

https://youtu.be/WrpQcFJOFQ4 , title: [ U.G. Krishnamurti on Parveen Babi ]
     January 1990.  Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erG

https://youtu.be/kKkBzHrWSVY , title: [ U.G. Krishnamurti - You Can Never Be Permanently Happy, Nor Have Permanent Pleasure ]
     After a date check U.G. blasts in and on. “Never trust a man who says he is conc

https://youtu.be/1elfvTKqdVk , title: [ U.G. Krishnamurti - You Have Lost Your Properties… ]
     Short clip, hearable but not good audio. Just for the record.  Books on U.G. Kri

https://youtu.be/aP-JiJxcz3o , title: [ U.G. Krishnamurti – What Is the Mind You Are Talking Of? ]
     U.G. mentions how he walked and talked with cobra snakes…  Books on U.G. Krishna

https://youtu.be/r_b4LoxR0Mk , title: [ U.G. Krishn

In [11]:
# Search for videos from channel by channel ID
host = "www.googleapis.com"
path = "youtube/v3/search"
query = {
    "key": GOOGLE_API_KEY,
    "channelId": "UCRLQIvQcoF41xZdrtKMXSvg",
    "maxResults": 20,
    "part": ["snippet"],
    "regionCode": "CA",
    "safeSearch": "none",
    "type": "video"
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
    if res.status_code != 200:
        pp.pprint(payload)
except Exception as e:
    print("Exception:", e)

# each search result (in items) is one of
# https://developers.google.com/resources/api-libraries/documentation/youtube/v3/csharp/latest/classGoogle_1_1Apis_1_1YouTube_1_1v3_1_1Data_1_1SearchResult.html
print()
print("===Content===")
print("pageInfo:")
for k, v in payload["pageInfo"].items():
    print("    ", k, v)
print("regionCode", payload["regionCode"])
print("prevPageToken:", payload.get("prevPageToken", None))
print("nextPageToken:", payload.get("nextPageToken", None))

print()
print("===Item 0===")
item = payload["items"][0]
videoId = item["id"]["videoId"]
snippet = item["snippet"]
title = snippet["title"]
description = snippet["description"]
print("title:", title)
print("videoId:", videoId)
print("description:", description.replace("\n"," ")[:200])
# each snippet is one of
snippet = item["snippet"]
print(snippet["publishedAt"])
# Date and time of publication in format:
# https://www.w3.org/TR/NOTE-datetime
# strptime() explained in:
# https://docs.python.org/3/library/time.html#time.strftime
publishedAt = datetime.strptime(snippet["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
print("publishedAt:", publishedAt)

channelTitle = snippet["channelTitle"]
channelId = snippet["channelId"]
print("From channel", channelTitle, ", channelId:", channelId)

Status code: 200

===Content===
pageInfo:
     totalResults 320
     resultsPerPage 20
regionCode CA
prevPageToken: None
nextPageToken: CBQQAA

===Item 0===
title: U.G. Krishnamurti - You&#39;re Doing Nothing!
videoId: gtXvDttcKgg
description: Short and to the point clip of U.G. slicing up the man's ideas. One has got to love the word 'nothing' after all watching some videos ...
2015-12-23T07:42:54Z
publishedAt: 2015-12-23 07:42:54
From channel Lectures Beyond Beyond , channelId: UCRLQIvQcoF41xZdrtKMXSvg


In [89]:
# Get all the videos from a channel using search.list, given channel ID
pageToken = None
count = 0
items = []
while True:
    host = "www.googleapis.com"
    path = "youtube/v3/search"
    query = {
        "key": GOOGLE_API_KEY,
        "channelId": "UCRLQIvQcoF41xZdrtKMXSvg",
        "maxResults": 50,
        "part": "snippet",
        "regionCode": "CA",
        "safeSearch": "none",
        "type": "video",
        "pageToken": pageToken
    }
    query = query_to_str(query)
    query = f"https://{host}/{path}?{query}"
    try:
        res = requests.get(query)
        payload = json.loads(res.content)
        items.extend(payload["items"])
        count += 1
        print("o ", end=" ")
        try:
            pageToken = payload["nextPageToken"]
        except KeyError:
            break    
    except Exception as e:
        print("Exception:", e)

print()
print(f"Made {count} requests")
print(f"Got {len(items)} videos")

o  o  o  o  o  o  o  
Made 7 requests
Got 309 videos


In [90]:
# List all the videos we go earlier, and print the links to the videos
should_print_link = True
for item in items:
    videoId = item["id"]["videoId"]
    snippet = item["snippet"]
    title = snippet["title"]
    description = snippet["description"]
    description = description.replace("\n"," ")[:80]
    if should_print_link:
        print(f"https://youtu.be/{videoId}", end=" ")
        print(f", title: [ {title} ]")
    else:
        print(f"videoId: {videoId} , title: [ {title} ]")
    print("    ", description)
    print()

https://youtu.be/gtXvDttcKgg , title: [ U.G. Krishnamurti - You&#39;re Doing Nothing! ]
     Short and to the point clip of U.G. slicing up the man's ideas. One has got to l

https://youtu.be/OUPWLRrkN1Y , title: [ U.G. Krishnamurti - Your Expectation, Your Delusion ]
     Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage

https://youtu.be/6deGa7wkidw , title: [ U.G. Krishnamurti - Thoughts, Language &amp; Energy ]
     Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage

https://youtu.be/knm0vDweuOc , title: [ U.G. Krishnamurti - Negative Philosophy ]
     Interview excerpt with Terry Newland. "I don't trust a man who wants to be selfl

https://youtu.be/VxvPQwhzKpE , title: [ U.G. Krishnamurti - Material &amp; Spiritual ]
     Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage

https://youtu.be/TiMFeBce_2Y , title: [ U.G. Krishnamurti - Energy Levels ]
     Books on U.G. Krishnamurti: Mind is a M

In [79]:
# Get subscription data from a user/channel
# possible to get 403, since subscriptions are private
# need to use InstalledAppFlow
host = "www.googleapis.com"
path = "youtube/v3/subscriptions"
part = "channelId=UCSk6p_put_fIDe7vXdw_PDQ"
query = f"https://{host}/{path}?{part}&key={GOOGLE_API_KEY}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)
payload

Status code: 403


{'error': {'code': 403,
  'message': 'The requester is not allowed to access the requested subscriptions.',
  'errors': [{'message': 'The requester is not allowed to access the requested subscriptions.',
    'domain': 'youtube.subscription',
    'reason': 'subscriptionForbidden'}]}}

## OAuth 2.0

In [2]:
# Get OAuth 2.0 authenticated session to get my YT private data.

# Based on:
# https://github.com/youtube/api-samples

# The CLIENT_SECRETS_FILE specifies the name of a file that contains
# OAuth 2.0 client ID and client secret from Google Cloud Console at
# https://cloud.google.com/console

# Need to enable the YouTube Data API for your project.
# See: https://developers.google.com/youtube/v3/guides/authentication
CLIENT_SECRETS_FILE = '../.secrets/client_secret.json'

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account.
SCOPES = ['https://www.googleapis.com/auth/youtube']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
credentials = flow.run_console()
youtube = build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517970623738-ctnfm8dhn55jbmrtsn1kmb4i68isl2ah.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=4nJWLDWfSokA5UDaaC1QdC0Pe4nA73&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWgol6Xnpf2oe2COjSeXImyjF9mEer6BtpdY3TEYsSXItHzx4KmYekQ


In [200]:
# Use session to query for my list of subscribers
# See: https://developers.google.com/youtube/v3/docs/subscriptions/list
payload = youtube.subscriptions().list(
    part="id,contentDetails,snippet,subscriberSnippet",
    maxResults=50,
    mine=True,
#     pageToken="CDIQAA"
).execute()

In [201]:
# Print the results of subscriptions.list
# See also:
# https://developers.google.com/resources/api-libraries/documentation/youtube/v3/csharp/latest/classGoogle_1_1Apis_1_1YouTube_1_1v3_1_1Data_1_1SubscriptionContentDetails.html
print("pageInfo:", payload["pageInfo"])
print("prevPageToken:", payload.get("prevPageToken", None))
print("nextPageToken:", payload.get("nextPageToken", None))

item = payload["items"][0]
subscriptionId = item["id"]
snippet = item["snippet"]
contentDetails = item["contentDetails"]
subscriberSnippet = item["subscriberSnippet"]

print()
print("contentDetails:")
# ActivityType : The type of activity this subscription is for (only uploads, everything).
# NewItemCount : The number of new items in the subscription since its content was last read.
# TotalItemCount : The approximate number of items that the subscription points to.
pp.pprint(contentDetails)
print("Subscription ID:", subscriptionId)
print("channelId:", snippet["resourceId"]["channelId"])
print("title:", snippet["title"])
print("description:")
print("[[[", description.replace("\n", " ")[:200], "]]]")
# pp.pprint(snippet)
# pp.pprint(subscriberSnippet)

pageInfo: {'totalResults': 530, 'resultsPerPage': 50}
prevPageToken: None
nextPageToken: CDIQAA

contentDetails:
{'activityType': 'all', 'newItemCount': 0, 'totalItemCount': 353}
Subscription ID: Hi7kfV3Wm_C0r2SC_Pso2898maAsQbjHU9z0PHvoDww
channelId: UC1pdbN6j_txoNhXVWVtouuQ
title: World Armwrestling League
description:
[[[ Part 1: The Moon and Stars ━━━━━━━━━━━━━━━━━━━━━━━━━━ Follow me on Twitter! https://twitter.com/realMax0r Patreon: https://www.patreon.com/max0r Join the Max0r Discord! https://discord.gg/max0r Twitch ]]]


In [3]:
# Get all the subscriptions
pageToken = None
count = 0
items = []
while True:
    payload = youtube.subscriptions().list(
        part="id,contentDetails,snippet",
        maxResults=50,
        mine=True,
        pageToken=pageToken,
    ).execute()
    items.extend(payload["items"])
    count += 1
    try:
        pageToken = payload["nextPageToken"]
    except KeyError:
        break
print(f"Made {count} requests")
print(f"Got {len(items)} subscriptions")

Made 11 requests
Got 531 subscriptions


In [4]:
# List all the subscriptions, and print the links to the channels
should_print_link = False
for item in items:
    snippet = item["snippet"]
    contentDetails = item["contentDetails"]
    # total number of content/items in channel
    totalItemCount = contentDetails["totalItemCount"]
    channelId = snippet["resourceId"]["channelId"]
    title = snippet["title"]
    if should_print_link:
        print(f"https://youtube.com/channel/{channelId}", end="")
        print(f" , title: [ {title} ], {totalItemCount} videos")
    else:
        print(f"channelId: {channelId} , title: [ {title} ], {totalItemCount} videos")

channelId: UCsXV4t9YfUeDuZ276hE20Jg , title: [ Adiyogi ], 346 videos
channelId: UC1pdbN6j_txoNhXVWVtouuQ , title: [ World Armwrestling League ], 353 videos
channelId: UCGGZ_POGmIWG1pQXTDzQv-g , title: [ Michelle Khare ], 189 videos
channelId: UCeUJFQ0D9qs6aVNyUt9fkeQ , title: [ The Armchair Historian ], 138 videos
channelId: UCMbvTF91O--bhwmg9dTMAPA , title: [ Lyle Forever ], 242 videos
channelId: UCrDQW9kAElm707c5z6d5r7Q , title: [ albie ], 62 videos
channelId: UCfgh3Ul_dG6plQ7rzuOLx-w , title: [ Max0r ], 81 videos
channelId: UC6JhS4GvWf3AJfOTfkrse2w , title: [ Simulation ], 1490 videos
channelId: UCvQ9wJKXJwcw_4tNaHHbWdA , title: [ Speak The Truth ], 291 videos
channelId: UCoydhtfFSk1fZXNRnkGnneQ , title: [ ESOTERICA ], 167 videos
channelId: UCE9htqwsu0II6KrpVKdrXHw , title: [ Zuck That ], 18 videos
channelId: UCHFYJPHg6mO0uEMhIla8lFQ , title: [ Content Savant ], 126 videos
channelId: UCkySD00cmDWYHXA31hqRYRw , title: [ Gennady Korotkevich ], 20 videos
channelId: UCOKfTeI3E2VVOqqV_KP

## Website Crawling

In [51]:
# search YT video website for metadata, etc...
links = [
    # Page Rank
    "https://www.youtube.com/watch?v=JGQe4kiPnrU",
    # MentalOutlaw - Archinstall - Arch Linux Made Easy
    "https://www.youtube.com/watch?v=G-mLyrHonvU"
]
link = links[1]
try:
    resp = urllib.request.urlopen(link)
    
    # Slower and returns less meta content 
    # resp = requests.get(link)
    
    try:
        print("Status code:", resp.getcode())
        content = resp.read().decode("utf-8")
    except AttributeError as e:
        print("Status code:", resp.status_code)
        content = resp.content
    soup = BeautifulSoup(content, 'html')
except Exception as e:
    print("Exception:", e)

print()
print("===Content===")
print("tags in head:")
print(*get_children_names(soup.html.head))

print()
print("tags in body:")
print(*get_children_names(soup.html.body))

print()
m = soup.html.head.find("meta", {"property": "og:title"})
print("title:", m["content"])

m = soup.html.head.find("meta", {"property": "og:description"})
print("description:", m["content"])

m = soup.html.head.find("meta", {"property": "og:url"})
print("url:", m["content"])

m = soup.html.head.find("meta", {"property": "og:image"})
print("thumbnail url:", m["content"])

m = soup.find("meta", {"itemprop": "uploadDate"})
print("upload date:", m["content"])

m = soup.find("meta", {"itemprop": "videoId"})
print("video ID:", m["content"])

m = soup.find("meta", {"itemprop": "channelId"})
print("channel ID:", m["content"])

# soup.html.head.findAll("meta")

Status code: 200

===Content===
tags in head:
meta script script script link link link link link script script script script script script script script script script script script link script link link link link style style style style style meta link link script script link link link title meta meta meta link link link link link link meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta

tags in body:
div script div script script script script script script script iframe ytd-app script script script script script script script script script script div script script script script script script script script

title: Archinstall - Arch Linux Made Easy
description: Now you can also become a Linux elitist  by using archinstall to easily deploy an Arch Linux machine.Song in vid

In [52]:
links = [
    "https://www.youtube.com/playlist?list=UUoydhtfFSk1fZXNRnkGnneQ"
]
link = links[0]
try:
    resp = urllib.request.urlopen(link)
    
    # Slower and returns less meta content 
    # resp = requests.get(link)
    
    try:
        print("Status code:", resp.getcode())
        content = resp.read().decode("utf-8")
    except AttributeError as e:
        print("Status code:", resp.status_code)
        content = resp.content
    soup = BeautifulSoup(content, 'html')
except Exception as e:
    print("Exception:", e)

Status code: 200


In [63]:
print()
print("===Content===")
print("tags in head:")
print(*get_children_names(soup.html.head))

print()
print("tags in body:")
print(*get_children_names(soup.html.body))


===Content===
tags in head:
meta script script script link link link link link script script script script script script script script script script script script script script script script link script link style style style style style meta link link

tags in body:
script script script iframe ytd-app script script script script script script script script script script script script script link link title link meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta meta link link link script link div script script script


'<body dir="ltr">\n <script nonce="hZNEl45WBdhPdcFIJyyz3A">\n  if (window.ytcsi) {window.ytcsi.tick(\'bs\', null, \'\');}\n </script>\n <script nonce="hZNEl45WBdhPdcFIJyyz3A">\n  ytcfg.set(\'initialBodyClientWidth\', document.body.clientWidth);\n </script>\n <script nonce="hZNEl45WBdhPdcFIJyyz3A">\n  if (window.ytcsi) {window.ytcsi.tick(\'ai\', null, \'\');}\n </script>\n <iframe name="passive_signin" src="https://accounts.google.com/ServiceLogin?service=youtube&amp;uilel=3&amp;passive=true&amp;continue=https%3A%2F%2Fwww.youtube.com%2Fsignin%3Faction_handle_signin%3Dtrue%26app%3Ddesktop%26hl%3Den-GB%26next%3D%252Fsignin_passive%26feature%3Dpassive&amp;hl=en-GB" style="display: none">\n </iframe>\n <ytd-app>\n  <ytd-masthead class="shell" id="masthead" logo-type="YOUTUBE_LOGO" slot="masthead">\n   <div class="ytd-searchbox-spt" id="search-container" slot="search-container">\n   </div>\n   <div class="ytd-searchbox-spt" id="search-input" slot="search-input">\n    <input autocapitalize="n

In [65]:

# print(soup.body.prettify())